<a href="https://colab.research.google.com/github/IhorViskrivets/python_projects/blob/main/A_B_Test_Significance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade google-cloud-bigquery

In [ ]:
from google.colab import auth
from google.cloud import bigquery
from google.colab import files
import pandas as pd
from statsmodels.stats.proportion import proportions_ztest

In [ ]:
auth.authenticate_user()
client = bigquery.Client(project="data-analytics-mate")

# SQL-запит
query = """
WITH
  session_info AS (
  SELECT
    s.date,
    s.ga_session_id,
    sp.country,
    sp.device,
    sp.continent,
    sp.channel,
    ab.test,
    ab.test_group
  FROM
    `DA.ab_test` ab
  JOIN
    `DA.session` s
  ON
    ab.ga_session_id = s.ga_session_id
  JOIN
    `DA.session_params` sp
  ON
    sp.ga_session_id = ab.ga_session_id ),
  session_with_orders AS (
  SELECT
    si.date,
    si.country,
    si.device,
    si.continent,
    si.channel,
    si.test,
    si.test_group,
    COUNT(DISTINCT o.ga_session_id) AS session_with_orders
  FROM
    `DA.order` o
  JOIN
    session_info si
  ON
    o.ga_session_id = si.ga_session_id
  GROUP BY
    si.date,
    si.country,
    si.device,
    si.continent,
    si.channel,
    si.test,
    si.test_group ),
  events AS (
  SELECT
    si.date,
    si.country,
    si.device,
    si.continent,
    si.channel,
    si.test,
    si.test_group,
    ep.event_name,
    COUNT(ep.ga_session_id) AS event_cnt
  FROM
    `DA.event_params` ep
  JOIN
    session_info si
  ON
    ep.ga_session_id = si.ga_session_id
  GROUP BY
    si.date,
    si.country,
    si.device,
    si.continent,
    si.channel,
    si.test,
    si.test_group,
    ep.event_name ),
  session AS (
  SELECT
    si.date,
    si.country,
    si.device,
    si.continent,
    si.channel,
    si.test,
    si.test_group,
    COUNT(DISTINCT si.ga_session_id) AS session_cnt
  FROM
    session_info si
  GROUP BY
    si.date,
    si.country,
    si.device,
    si.continent,
    si.channel,
    si.test,
    si.test_group ),
  account AS (
  SELECT
    si.date,
    si.country,
    si.device,
    si.continent,
    si.channel,
    si.test,
    si.test_group,
    COUNT(DISTINCT acs.ga_session_id) AS new_account_cnt
  FROM
    `DA.account_session` acs
  JOIN
    session_info si
  ON
    acs.ga_session_id = si.ga_session_id
  GROUP BY
    si.date,
    si.country,
    si.device,
    si.continent,
    si.channel,
    si.test,
    si.test_group )
SELECT
  session_with_orders.date,
  session_with_orders.country,
  session_with_orders.device,
  session_with_orders.continent,
  session_with_orders.channel,
  session_with_orders.test,
  session_with_orders.test_group,
  'session with orders' AS event_name,
  session_with_orders.session_with_orders AS value
FROM
  session_with_orders
UNION ALL
SELECT
  events.date,
  events.country,
  events.device,
  events.continent,
  events.channel,
  events.test,
  events.test_group,
  event_name,
  event_cnt AS value
FROM
  events
UNION ALL
SELECT
  session.date,
  session.country,
  session.device,
  session.continent,
  session.channel,
  session.test,
  session.test_group,
  'session' AS event_name,
  session_cnt AS value
FROM
  session
UNION ALL
SELECT
  account.date,
  account.country,
  account.device,
  account.continent,
  account.channel,
  account.test,
  account.test_group,
  'new account' AS event_name,
  new_account_cnt AS value
FROM
  account
"""

query_job = client.query(query)  # виконання SQL-запиту
results = query_job.result()
df = results.to_dataframe() # перетворення результатів на датафрейм
df.head()

,date,country,device,continent,channel,test,test_group,event_name,value
0,2020-11-01,Kuwait,mobile,Asia,Organic Search,2,2,session,1
1,2020-11-01,Nepal,desktop,Asia,Paid Search,2,1,session,1
2,2020-11-01,Costa Rica,desktop,Americas,Direct,2,2,session,1
3,2020-11-01,Serbia,desktop,Europe,Paid Search,2,1,session,1
4,2020-11-01,Paraguay,desktop,Americas,Social Search,2,2,session,1


In [ ]:
# список метрик
metrics = ["add_payment_info",
    "add_shipping_info",
    "begin_checkout",
    "new account"
    ]

results = [] # створюємо список результатів
tests = df["test"].unique() # унікальні номери тестів
# обраховуємо конверсійні метрики за допомогою циклів
for test in tests: # йдемо циклом по тестам
  test_df = df[df["test"] == test]
  for metric in metrics: # йдемо циклом по метрикам
    for group in [1, 2]: # йдемо циклом по групам
      group_df = test_df[test_df["test_group"] == group]
      metric_sum = group_df[group_df["event_name"] == metric]["value"].sum()
      sessions_sum = group_df[group_df["event_name"] == "session"]["value"].sum()
      if group == 1:
        metric_A = metric_sum
        sessions_A = sessions_sum
      else:
        metric_B = metric_sum
        sessions_B = sessions_sum
    if sessions_A != 0:
      conversion_A = metric_A / sessions_A
    else:
      conversion_A = 0
    if sessions_B != 0:
      conversion_B = metric_B / sessions_B
    else:
      conversion_B = 0
    if conversion_A > 0:
      conversion_uplift = (conversion_B - conversion_A) / conversion_A * 100
    else:
      conversion_uplift = 0
    # проводимо тест пропорцій
    z_stat, p_value = proportions_ztest([metric_A, metric_B], [sessions_A, sessions_B])
    # додаємо результати у створений раніше список
    results.append({"test_number": test,
                    "metric": metric,
                    "conversion_rate_A": conversion_A,
                    "conversion_rate_B": conversion_B,
                    "uplift_percent": conversion_uplift,
                    "z_stat": z_stat,
                    "p_value": p_value,
                    "significant": p_value < 0.05})

final_results = pd.DataFrame(results) # перетворюємо результати на датафрейм

final_results # перевіряємо



,test_number,metric,conversion_rate_A,conversion_rate_B,uplift_percent,z_stat,p_value,significant
0,2,add_payment_info,0.046290,0.047946,3.576911,-1.240994,0.214608,False
1,2,add_shipping_info,0.068724,0.069859,1.650995,-0.709557,0.477979,False
2,2,begin_checkout,0.084168,0.085841,1.988164,-0.952898,0.340642,False
3,2,new account,0.082252,0.083274,1.241934,-0.588793,0.556000,False
4,1,add_payment_info,0.043825,0.049322,12.542021,-3.924884,0.000087,True
5,1,add_shipping_info,0.066884,0.071272,6.560481,-2.603571,0.009226,True
6,1,begin_checkout,0.083418,0.088974,6.660587,-2.978783,0.002894,True
7,1,new account,0.084278,0.081451,-3.354299,1.542883,0.122859,False
8,4,add_payment_info,0.035507,0.034249,-3.541234,1.571106,0.116158,False
9,4,add_shipping_info,0.048801,0.047137,-3.411125,1.785795,0.074132,False


In [ ]:
final_results.to_excel("ab_test.xlsx", index=False)
files.download("ab_test.xlsx")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Підсумки

### Тест 1

Метрики add_payment_info, add_shipping_info та begin_checkout статистично значущо покращилися.

Кількість нових акаунтів знизилася, але результати статистично незначущі.

### Тест 2

Всі чотири основні метрики покращили показник конверсії, але жоден результат не здобув статистичної значущості.

### Тест 3

Метрика begin_checkout статистично значущо знизилася. Інші показники не отримали статистичної значущості.

### Тест 4

Всі метрики показали від'ємний результат. З них статистично значущо знилися лише begin_checkout та new account.

Візуалізацію рещультатів можна побачити на [дешборді](https://public.tableau.com/app/profile/ihor.viskrivets/viz/ABTestResults_17602733254540/ABTestResults#1) Tableau Public